In [1]:
# ============================================================
# Cell 1 — Imports, paths, and logging helper
# ============================================================

import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

# Paths
METADATA_PARQUET = Path("data/interim/clinical_trials_metadata.parquet")
MAUDE_SUMMARY = Path("data/external/maude_safety_summary.csv")

PHASE_COST_FACTORS = Path("data/external/trial_phase_cost_factors.csv")
REGION_MULTIPLIERS = Path("data/external/region_enrollment_multipliers.csv")

def log(msg: str) -> None:
    print(msg)

Cell 1 — Load ClinicalTrials.gov Metadata and Reference Tables
This cell initializes the scenario-building workflow by loading all prerequisite data sources:
ClinicalTrials.gov metadata (clinical_trials_metadata.parquet), produced in the ingestion notebook.
Cost reference tables, including:
-trial_phase_cost_factors.csv (typical per-phase inflation factors)
-region_enrollment_multipliers.csv (geographic feasibility multipliers)
These reference files provide the backbone for estimating trial-level costs and feasibility scores.The goal is to keep raw ingestion and scenario modeling cleanly separated so that optimization notebooks downstream can always rely on this preprocessed, structured data layer.

In [2]:
# ============================================================
# Cell 2 — Load CT.gov trial metadata (full corpus)
# ============================================================

if not METADATA_PARQUET.exists():
    raise FileNotFoundError(
        f"Expected {METADATA_PARQUET} from Notebook 01, but it was not found."
    )

trials_meta = pd.read_parquet(METADATA_PARQUET)
log(f"[Cell 2] Loaded trials_meta from {METADATA_PARQUET} with shape {trials_meta.shape}")
trials_meta.head()

[Cell 2] Loaded trials_meta from data/interim/clinical_trials_metadata.parquet with shape (557292, 14)


,nct_id,brief_title,official_title,overall_status,phase,conditions,interventions,enrollment,location_countries,lead_sponsor,size_bytes,last_modified,folder,s3_key
0,NCT00000102,Congenital Adrenal Hyperplasia: Calcium Channe...,None,Completed,Phase 1/Phase 2,[Congenital Adrenal Hyperplasia],[Nifedipine],None,[United States],National Center for Research Resources (NCRR),4217,2025-11-14 10:58:29+00:00,NCT0000xxxx,clinical-trials-data/raw/NCT0000xxxx/NCT000001...
1,NCT00000104,Does Lead Burden Alter Neuropsychological Deve...,None,Completed,None,[Lead Poisoning],[ERP measures of attention and memory],None,[United States],National Center for Research Resources (NCRR),4357,2025-11-14 10:58:29+00:00,NCT0000xxxx,clinical-trials-data/raw/NCT0000xxxx/NCT000001...
2,NCT00000105,Vaccination With Tetanus and KLH to Assess Imm...,Vaccination With Tetanus Toxoid and Keyhole Li...,Terminated,None,[Cancer],"[Intracel KLH Vaccine, Biosyn KLH, Montanide I...",112,[United States],"Masonic Cancer Center, University of Minnesota",11623,2025-11-14 10:58:29+00:00,NCT0000xxxx,clinical-trials-data/raw/NCT0000xxxx/NCT000001...
3,NCT00000106,41.8 Degree Centigrade Whole Body Hyperthermia...,None,Unknown status,N/A,[Rheumatic Diseases],[Whole body hyperthermia unit],None,[United States],National Center for Research Resources (NCRR),5572,2025-11-14 10:58:29+00:00,NCT0000xxxx,clinical-trials-data/raw/NCT0000xxxx/NCT000001...
4,NCT00000107,Body Water Content in Cyanotic Congenital Hear...,None,Completed,None,"[Heart Defects, Congenital]",[],None,[United States],National Center for Research Resources (NCRR),3312,2025-11-14 10:58:29+00:00,NCT0000xxxx,clinical-trials-data/raw/NCT0000xxxx/NCT000001...


Cell 2 — Normalize Text Fields for Clustering and Grouping
This cell standardizes several key text fields to improve consistency across trials:
-lead_sponsor → normalized into a lowercase, slug-like version (lead_sponsor_norm)
-Country lists converted into strings where appropriate
-Otional cleaning applied to phases and statuses
Normalizing these fields produces stable identifiers that support grouping, pivoting, filtering, and later optimization tasks.

In [3]:
# ============================================================
# Cell 3 — Load phase cost factors and region enrollment multipliers
# ============================================================

if not PHASE_COST_FACTORS.exists():
    raise FileNotFoundError(
        f"Expected {PHASE_COST_FACTORS}. "
        "Run 00_generate_reference_tables.py first."
    )

if not REGION_MULTIPLIERS.exists():
    raise FileNotFoundError(
        f"Expected {REGION_MULTIPLIERS}. "
        "Run 00_generate_reference_tables.py first."
    )

phase_costs = pd.read_csv(PHASE_COST_FACTORS)
region_mults = pd.read_csv(REGION_MULTIPLIERS)

log(f"[Cell 3] phase_costs shape: {phase_costs.shape}")
display(phase_costs.head())

log(f"[Cell 3] region_mults shape: {region_mults.shape}")
display(region_mults.head())

[Cell 3] phase_costs shape: (9, 2)


,phase,cost_multiplier
0,Early Phase 1,0.8
1,Phase 1,1.0
2,Phase 1/Phase 2,1.5
3,Phase 2,2.0
4,Phase 2/Phase 3,3.0


[Cell 3] region_mults shape: (7, 2)


,region_label,enrollment_multiplier
0,North America,1.00
1,Western Europe,0.95
2,Eastern Europe,1.05
3,Asia-Pacific,1.10
4,Latin America,0.90


Cell 3 — Map Countries to Global Regions

This cell maps each trial's location_countries into broader region labels such as:
-North America
-Europe
-East Asia / Pacific
-Middle East / Africa
-Latin America
Each trial may recruit in multiple countries, so this step consolidates geographic footprints into a single region used for coarse-grained feasibility modeling and downstream scenario definitions.
If no country information is present, trials default to "Unknown".

In [4]:
# ============================================================
# Cell 4 — Optional MAUDE safety merge (sponsor-level)
# ============================================================

def normalize_name(name: str) -> str | None:
    """Simple normalization for sponsor/manufacturer names."""
    if pd.isna(name):
        return None
    return " ".join(str(name).upper().split())

if MAUDE_SUMMARY.exists():
    log(f"[Cell 4] MAUDE summary found at {MAUDE_SUMMARY}; loading for safety merge.")
    maude_safety = pd.read_csv(MAUDE_SUMMARY)
    log(f"[Cell 4] maude_safety shape: {maude_safety.shape}")
    display(maude_safety.head())

    # Normalized sponsor on trials
    trials_meta["lead_sponsor_norm"] = trials_meta["lead_sponsor"].apply(normalize_name)

    # Figure out which column holds the manufacturer name in MAUDE summary
    if "manufacturer_normalized" in maude_safety.columns:
        manufacturer_col = "manufacturer_normalized"
    elif "manufacturer" in maude_safety.columns:
        manufacturer_col = "manufacturer"
    else:
        raise KeyError(
            "MAUDE summary must have either 'manufacturer_normalized' or 'manufacturer' column."
        )

    maude_safety["manufacturer_norm"] = maude_safety[manufacturer_col].apply(normalize_name)

    # Collect columns to keep from MAUDE
    safety_cols = ["manufacturer_norm"]
    for col in ["total_events", "serious_events", "deaths", "safety_score"]:
        if col in maude_safety.columns:
            safety_cols.append(col)

    maude_for_merge = maude_safety[safety_cols].copy()
    log("[Cell 4] Example MAUDE rows used for merge:")
    display(maude_for_merge.head())

    # Left-join trials to safety on normalized name
    trials_with_safety = trials_meta.merge(
        maude_for_merge,
        how="left",
        left_on="lead_sponsor_norm",
        right_on="manufacturer_norm",
        suffixes=("", "_maude"),
    )

    log(f"[Cell 4] trials_with_safety shape after MAUDE merge: {trials_with_safety.shape}")

    match_frac = (~trials_with_safety["manufacturer_norm"].isna()).mean()
    log(f"[Cell 4] Fraction of trials with matched MAUDE manufacturer: {match_frac:.1%}")

else:
    log(f"[Cell 4] No MAUDE summary at {MAUDE_SUMMARY}; proceeding without safety features.")
    # Still create lead_sponsor_norm so later code works
    trials_with_safety = trials_meta.copy()
    trials_with_safety["lead_sponsor_norm"] = trials_with_safety["lead_sponsor"]


[Cell 4] No MAUDE summary at data/external/maude_safety_summary.csv; proceeding without safety features.


Cell 4 — Optional Merge of Safety Signals (MAUDE)

This cell attempts to merge per-trial safety indicators derived from the FDA MAUDE dataset, if available.
The MAUDE summary file may contain fields such as:
-total_events
-serious_events
-deaths
-safety_score
If present, the notebook merges these columns onto each trial using nct_id.
If not found, the cell proceeds silently and continues without safety augmentation.
This optionality allows the scenario framework to remain modular: additional safety, quality, and operational data can be layered on later without disrupting the main pipeline.

In [5]:
# ============================================================
# Cell 5 — Derive region labels and compute cost + feasibility
# ============================================================

def classify_region(countries):
    """
    Very simple, heuristic region classifier based on the list of
    location_countries. You can refine this later.
    """
    if not isinstance(countries, list) or len(countries) == 0:
        return "Global / Multi-Region"

    upper = [str(c).upper() for c in countries]

    if any(c in upper for c in ["UNITED STATES", "CANADA"]):
        return "North America"
    if any(c in upper for c in ["UNITED KINGDOM", "FRANCE", "GERMANY", "SPAIN", "ITALY"]):
        return "Western Europe"
    if any(c in upper for c in ["POLAND", "HUNGARY", "CZECH REPUBLIC", "ROMANIA"]):
        return "Eastern Europe"
    if any(c in upper for c in ["CHINA", "JAPAN", "SOUTH KOREA", "INDIA", "AUSTRALIA"]):
        return "Asia-Pacific"
    if any(c in upper for c in ["BRAZIL", "ARGENTINA", "MEXICO", "COLOMBIA", "CHILE"]):
        return "Latin America"
    if any(c in upper for c in ["SOUTH AFRICA", "EGYPT", "SAUDI ARABIA", "UAE"]):
        return "Middle East & Africa"

    return "Global / Multi-Region"

# 1) Region label from location_countries
trials_with_safety["region_label"] = trials_with_safety["location_countries"].apply(classify_region)

# 2) Merge phase cost factors
trials_with_safety = trials_with_safety.merge(
    phase_costs,
    how="left",
    left_on="phase",
    right_on="phase",
    suffixes=("", "_phase"),
)

# 3) Merge region enrollment multipliers
trials_with_safety = trials_with_safety.merge(
    region_mults,
    how="left",
    on="region_label",
    suffixes=("", "_region"),
)

log(f"[Cell 5] trials_with_safety shape after cost + region merges: {trials_with_safety.shape}")

# 4) Derived features:
#    - estimated_trial_cost: proportional to cost_multiplier (fallback = 1.0)
#    - enrollment_feasibility_score: enrollment_multiplier (fallback = 1.0)

trials_with_safety["cost_multiplier"] = trials_with_safety["cost_multiplier"].fillna(1.0)
trials_with_safety["enrollment_multiplier"] = trials_with_safety["enrollment_multiplier"].fillna(1.0)

trials_with_safety["estimated_trial_cost"] = trials_with_safety["cost_multiplier"]
trials_with_safety["enrollment_feasibility_score"] = trials_with_safety["enrollment_multiplier"]

log("[Cell 5] Sample of derived cost and feasibility features:")
trials_with_safety[[
    "nct_id",
    "phase",
    "region_label",
    "cost_multiplier",
    "enrollment_multiplier",
    "estimated_trial_cost",
    "enrollment_feasibility_score",
]].head()

[Cell 5] trials_with_safety shape after cost + region merges: (689038, 18)
[Cell 5] Sample of derived cost and feasibility features:


,nct_id,phase,region_label,cost_multiplier,enrollment_multiplier,estimated_trial_cost,enrollment_feasibility_score
0,NCT00000102,Phase 1/Phase 2,Global / Multi-Region,1.5,1.0,1.5,1.0
1,NCT00000104,None,Global / Multi-Region,1.0,1.0,1.0,1.0
2,NCT00000104,None,Global / Multi-Region,1.0,1.0,1.0,1.0
3,NCT00000105,None,Global / Multi-Region,1.0,1.0,1.0,1.0
4,NCT00000105,None,Global / Multi-Region,1.0,1.0,1.0,1.0


Cell 5 — Compute Cost Estimates and Feasibility Scores

This cell constructs two key modeled attributes for optimization:
-Estimated Trial Cost (estimated_trial_cost)
Computed using trial phase + region multipliers from the reference tables.
These reflect typical per-trial costs across global locations.
-Enrollment Feasibility Score (enrollment_feasibility_score)
A synthetic but structured metric projecting how difficult enrollment may be in a given region.
This score becomes a constraint in later optimization notebooks.
Both metrics ensure the final scenario table is suitable for resource allocation, portfolio simulation, and quantum optimization tasks.

In [6]:
# ============================================================
# Cell 6 — Build trials_scenarios and persist
# ============================================================

SCENARIOS_PARQUET = Path("data/interim/trials_scenarios.parquet")
SCENARIOS_SAMPLE_PARQUET = Path("data/interim/trials_scenarios_sample.parquet")

# Core columns for optimization
scenario_cols = [
    "nct_id",
    "brief_title",
    "overall_status",
    "phase",
    "conditions",
    "interventions",
    "location_countries",
    "lead_sponsor",
    "lead_sponsor_norm",
    "region_label",
    "estimated_trial_cost",
    "enrollment_feasibility_score",
]

# If MAUDE is added later, these will automatically appear
for col in ["total_events", "serious_events", "deaths", "safety_score"]:
    if col in trials_with_safety.columns:
        scenario_cols.append(col)

# Keep only columns that actually exist
scenario_cols = [c for c in scenario_cols if c in trials_with_safety.columns]

trials_scenarios = trials_with_safety[scenario_cols].copy()

log(f"[Cell 6] trials_scenarios shape: {trials_scenarios.shape}")
trials_scenarios.head()

# Persist full scenarios table
SCENARIOS_PARQUET.parent.mkdir(parents=True, exist_ok=True)
trials_scenarios.to_parquet(SCENARIOS_PARQUET, index=False)
log(f"[Cell 6] Wrote trials_scenarios to {SCENARIOS_PARQUET}")

# Also write a small sample for quick experiments / GitHub
sample_n = min(10000, len(trials_scenarios))
trials_scenarios_sample = trials_scenarios.sample(n=sample_n, random_state=42)
trials_scenarios_sample.to_parquet(SCENARIOS_SAMPLE_PARQUET, index=False)
log(f"[Cell 6] Wrote trials_scenarios_sample ({sample_n} rows) to {SCENARIOS_SAMPLE_PARQUET}")

[Cell 6] trials_scenarios shape: (689038, 12)
[Cell 6] Wrote trials_scenarios to data/interim/trials_scenarios.parquet
[Cell 6] Wrote trials_scenarios_sample (10000 rows) to data/interim/trials_scenarios_sample.parquet


Cell 6 — Assemble the Scenario Table and Persist to Disk

This cell consolidates all engineered fields into a single canonical dataset, trials_scenarios, including:
-Trial metadata (phase, status, sponsor, conditions)
-Normalized categorical fields
-Region assignment
-Estimated costs
-Feasibility scores
-Optional safety features
The full dataset is written to: data/interim/trials_scenarios.parquet
A smaller 10,000-row sample is also saved for:
-GitHub storage
-Quick experimentation
-Braket local debugging
-This sampled version ensures the repository remains lightweight while still providing a representative subset.

In [7]:
# ============================================================
# Cell 7 — Enforce one row per trial (nct_id)
# ============================================================

total_rows = len(trials_scenarios)
unique_nct = trials_scenarios["nct_id"].nunique(dropna=True)

log(f"[Cell 7] trials_scenarios total rows: {total_rows:,}")
log(f"[Cell 7] unique nct_id values:       {unique_nct:,}")

if total_rows != unique_nct:
    log("[Cell 7] WARNING: Detected multiple rows per nct_id. "
        "Collapsing to one row per trial using first occurrence.")

    # Simple, conservative collapse: keep the first row per nct_id.
    trials_scenarios = (
        trials_scenarios
        .sort_values("nct_id")
        .drop_duplicates(subset=["nct_id"], keep="first")
        .reset_index(drop=True)
    )

    total_rows2 = len(trials_scenarios)
    unique_nct2 = trials_scenarios["nct_id"].nunique(dropna=True)
    log(f"[Cell 7] After collapse → rows: {total_rows2:,}, unique nct_id: {unique_nct2:,}")

    # Re-write the Parquet artifacts with the cleaned table
    SCENARIOS_PARQUET = Path("data/interim/trials_scenarios.parquet")
    SCENARIOS_SAMPLE_PARQUET = Path("data/interim/trials_scenarios_sample.parquet")

    SCENARIOS_PARQUET.parent.mkdir(parents=True, exist_ok=True)
    trials_scenarios.to_parquet(SCENARIOS_PARQUET, index=False)
    log(f"[Cell 7] Re-wrote trials_scenarios to {SCENARIOS_PARQUET}")

    sample_n = min(10000, len(trials_scenarios))
    trials_scenarios_sample = trials_scenarios.sample(n=sample_n, random_state=42)
    trials_scenarios_sample.to_parquet(SCENARIOS_SAMPLE_PARQUET, index=False)
    log(f"[Cell 7] Re-wrote trials_scenarios_sample ({sample_n} rows) to {SCENARIOS_SAMPLE_PARQUET}")
else:
    log("[Cell 7] nct_id is already unique; no action needed.")


[Cell 7] trials_scenarios total rows: 689,038
[Cell 7] unique nct_id values:       557,292
[Cell 7] WARNING: Detected multiple rows per nct_id. Collapsing to one row per trial using first occurrence.
[Cell 7] After collapse → rows: 557,292, unique nct_id: 557,292
[Cell 7] Re-wrote trials_scenarios to data/interim/trials_scenarios.parquet
[Cell 7] Re-wrote trials_scenarios_sample (10000 rows) to data/interim/trials_scenarios_sample.parquet


Cell 7 — Deduplicate Trial Records to Ensure One Row per NCT ID

Some trials may appear multiple times due to multi-region expansion.
This cell enforces the rule:
One trial = one scenario row
It checks for duplicated nct_id values and, if found, collapses to the first occurrence.
The cleaned scenario table is then re-written to parquet.
This ensures downstream optimizers treat each trial as a single optimization candidate.

Final Summary — Scenario Preparation & Risk Feature Engineering
This notebook constructs the canonical scenario dataset used across all downstream optimization, simulation, and Braket workflows. Starting from the raw metadata generated in 01_clinical_trials_ingestion, we progressively enrich and structure the trial records into a format suitable for portfolio-level analytics and quantum resource allocation models.
Key Accomplishments in This Notebook
Loaded foundational metadata and reference tables
ClinicalTrials.gov trial-level metadata
Global region enrollment multipliers
Trial-phase cost factors
Standardized core fields
Normalized sponsor names
Harmonized phase and status fields
Cleaned location/country lists
Assigned geographic region labels
Mapped heterogeneous country lists into consolidated region categories
Ensured every trial has a region placeholder, even if unknown
Integrated optional MAUDE safety features (if present)
Safety events, serious events, mortality counts, and composite safety scores
Fully modular: safe to skip if MAUDE data is not yet loaded
Generated modeled cost and enrollment difficulty fields
Estimated trial cost via phase × region cost model
Enrollment feasibility score useful for scenario filtering and optimization constraint design
Built and persisted the unified scenario dataset
Full table: data/interim/trials_scenarios.parquet
Lightweight 10,000-row sample: trials_scenarios_sample.parquet
Ready for rapid local experimentation and cloud-scale optimization
Enforced one row per NCT ID
Deduplicated multi-region expansions
Guaranteed a clean, trial-level dataset for all downstream modeling
Next Steps in the Project Roadmap
The scenario dataset produced here becomes the foundational input for all subsequent notebooks. Next we will:
1. Integrate and stage MAUDE safety signals.
Even though this notebook supports them, we will load the true MAUDE dataset next and merge the computed safety metrics.
2. Develop the optimization-ready QUBO encodings
Notebook 03_qubo_model_construction will:
Define binary decision variables
Encode cost, feasibility, and sponsor constraints
Build the scalable QUBO matrices used by Braket simulators and hardware
3. Run experiments on AWS Braket
Using:
Local simulators

SV1
Real hardware (IonQ, QuEra, Rigetti)
4. Build evaluation and visualization notebooks
Comparing:
Classical baselines
QAOA variants
Hybrid optimizers
Hardware-vs-simulator performance
Outcome
With the final scenario table now complete, the project is ready to transition from data engineering → optimization modeling → quantum experimentation.